# Surface

## Index

- [Definition](#Definition)
- [Example : Plotting a Cylinder](#Example-:-Plotting-a-Cylinder)
- Creating new surface
    - Raw (for incomplete definition)
    - From metric
    - Surface Integrity Check
- [Evaluation](#Evaluation)
- [Transformations](#Transformations)
    - Translation
    - Rotation
    - Translation and Rotation
- Surface area calculation
- Poles
    - Example of the Sphere
- Numerical Surface
- Comments about precision

## Definition

A *surface* represented by the class `gsurface.surface.surface.Surface`, is defined using 2 parameters designated by $u$ and $v$. See complete definition on [Wikipedia](https://en.wikipedia.org/wiki/Surface_(mathematics)).

$$
\begin{align*}
\mathbf{S} : \mathbb{R}^2 &\to\mathbb{R}^3 \\
u, v &\mapsto \mathbf{S}(u, v)
\end{align*}
$$

## Example : Plotting a Cylinder

A cylinder of radius $r$ can be defined as following :

$$
\mathbf{Cyl}(u, v) = \begin{cases}
R cos(u) \\
R sin(u) \\
v
\end{cases}
$$

The `gsurface.surface.cylinder.Cylinder` class implements this surface. We can plot this surface using *mlab* and *mayavi_plot_surface* function.

In [1]:
from gsurface.surface import Cylinder
from gsurface.plotter import mayavi_plot_surface

cyl = Cylinder(R = 1.0)  # creating object

cyl.setlims(v_ll = 0, v_ul = 10)  # updating v bounds to [0, 10]

mesh = cyl.mesh(50, 50)  # creating u,v-mesh

smesh = cyl.build_surface(*mesh)  # generating surface points for this u,v-mesh

mayavi_plot_surface(smesh)  # plotting

**Symbols** :

- `cyl` is a *Surface:Cylinder object*
- `mesh` is as *2-dim np.ndarray* (50x50)
- `smesh` is as *3-dim np.ndarray* (50x50x3)

![mayavi_cylinder_ui.png](./pics/mayavi_cylinder_ui.png)

## Creating new surface

Here we are defining the surface with following parametric equations :

$$S (u, v)=\left[\begin{matrix}u - \cos{\left(v \right)}\\v \sin{\left(u \right)} + v\\u \sin{\left(u - v \right)} - v\end{matrix}\right]$$

In order to build our surface class, we need to find the Jacobian and the Hessians matrices :

$J = \left[\begin{matrix}
1 & \sin(v) \\
v \cos(u) & \sin(u) + 1 \\
u \cos(u - v) + \sin(u - v) & u \cos(u - v) - 1
\end{matrix}
\right]$

$H_x = \left[\begin{matrix}
0 & 0 \\
0 & \cos(v)
\end{matrix}\right]$

$H_y = \left[\begin{matrix}
v \sin(u) & \cos(u) \\
\cos(u) & 0
\end{matrix}\right]$

$H_z = \left[\begin{matrix}
- u \sin(u - v) + 2 \cos(u - v) & u \sin(u - v) - \cos(u - v) \\
u \sin(u - v) - \cos(u - v) & - u \sin(u - v)
\end{matrix}\right]$

Finally we can choose parameters limits, for example :
$u, v \in [-2, 2]^2$

## Evaluation

The point associated to the parametric position $(u, v)$ is $\mathbf{S}(u, v)$.

For the previous cylinder the point associaed to the parametric position $\left(\frac{pi}{2}, 1\right)$ is $\mathbf{Cyl}\left(\frac{pi}{2}, 1\right) = \begin{bmatrix} 0 \\ R \\ 1 \end{bmatrix}$

In [6]:
import numpy as np

S, J, H = cyl.eval(np.pi / 2, 1)

print(J)

[[-1.000000e+00  0.000000e+00]
 [ 6.123234e-17  0.000000e+00]
 [ 0.000000e+00  1.000000e+00]]


the `eval` function return 3 values $\mathbf{S}$ the position of the point on the surface (dimension $3$), $\mathbf{J}$ the [Jacobian](https://en.wikipedia.org/wiki/Jacobian_matrix_and_determinant) of $S$ (dimension $3\times2$) and [Hessian](https://en.wikipedia.org/wiki/Hessian_matrix) of $S_x$, $S_y$ and $S_z$ : $\mathbf{H} = \begin{bmatrix} \mathbf{H_x} & \mathbf{H_y} & \mathbf{H_z} \end{bmatrix}$ (dimension $(2\times2)$ each), all evaluated on the parametric position $(u, v)$.

These 3 values are gathered in the tuple `gsurface.types.SJH` of the library.

## Transformations

### Translation

### Rotation

### Rotation and Translation

In [1]:
import numpy as np

from gsurface.surface import Surface, SJH

In [20]:
class NewSurface(Surface):
    def __init__(self):
        # initializing base surface and setting limits
        super().__init__(
            plimits=np.array([
                [-2.0, 2.0],
                [-2.0, 2.0]
            ])
        )

    # this function must return a tuple of S, J, and Hxyz matrices
    # by using the buildMetric function we can simplify the definition
    # not filled values are set to 0.0
    def _definition(self, u: float, v: float) -> SJH:        
        return self.buildMetric(
            # S
            x=u - np.cos(v),
            y=v*np.sin(u) + v,
            z=np.sin(u - v)*u - v,
            
            # J
            dux=1,
            duy=v*np.cos(u),
            duz=u*np.cos(u - v) + np.sin(u - v),

            dvx=np.sin(v),
            dvy=np.sin(u) + 1,
            dvz=-u*np.cos(u - v) - 1,

            # Hx
            dvvx = np.cos(v),

            # Hy
            duuy=-v*np.sin(u),
            duvy=np.cos(u),

            # Hz
            duuz=-u*np.sin(u - v) + 2*np.cos(u - v),
            duvz=u*np.sin(u - v) - np.cos(u - v),
            dvvz=-u*np.sin(u - v),
        )
        

Check surface consistancy

In [3]:
surface = NewSurface()

result = surface.check_verbose(20, 20)

print(f"Surface model valid ? {result}")

=== Checking surface definition ===
U 20 points from -2.0 to 2.0
V 20 points from -2.0 to 2.0
Tolerance : 1.0E-07
index[0 : xi] max err = 4.2E-08 [True]
index[1 : yi] max err = 5.2E-08 [True]
index[2 : zi] max err = 6.5E-08 [True]
index[3 : duxi] max err = 0.0E+00 [True]
index[4 : dvxi] max err = 1.1E-08 [True]
index[5 : duyi] max err = 2.8E-08 [True]
index[6 : dvyi] max err = 1.4E-08 [True]
index[7 : duzi] max err = 5.3E-08 [True]
index[8 : dvzi] max err = 6.9E-08 [True]
Surface model valid ? True


Create model

In [4]:
from gsurface import SurfaceGuidedMassSystem, Tyi, build_s0
from gsurface.forces import ViscousFriction, SpringForce

model = SurfaceGuidedMassSystem(
    surface=surface,
    s0=build_s0(u0=2.5, du0=-8.0, dv0=5.0),
    m=1.0,
    forces=[
        SpringForce(10.0),
        ViscousFriction(1.0)
    ]
)

In [5]:
from gsurface.plotter import mayavi_plot_surfaces, SurfacePlot

surface  = NewSurface().multlims(1.5)

surface_mesh = surface.build_surface(*surface.mesh(50, 50))

time = np.linspace(0, 10, 1000)

data = model.solve(time)

solutions = model.solutions(data, time)

# animate does not work with jupyter notebooks
mayavi_plot_surfaces([
    SurfacePlot(surface_mesh, trajectory=solutions[Tyi], animate=False)
])

![New Surface Trajectory](../render/NewSurfaceTraj.png)